In [ ]:
import torch

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as pl
import scipy.signal as sg

pl.style.use('fivethirtyeight')
mpl.rcParams['axes.facecolor']='white'  
mpl.rcParams['figure.facecolor'] = '1'

from scipy.ndimage  import gaussian_filter1d

import json

In [ ]:
from masking import *
from latencies import *
from excitation import *
from deconv import *
from ur import *
from tuning import *
from test import *
from ur import *

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

### Simulate CAPs

In [ ]:
#masking degree function
#mdFunc=SigmoidMaskingDegreeFunction(30, 2*1/10., requires_grad=False)
mdFunc=WeibullCDF_IOFunc(0, 30, 7)
#latencies
lat0=Eggermont1976clickLatencies80dB #latencies

#Unitary response
ur0=URWang1979m
t=np.linspace(5e-4, 10e-3, num=500)
u00=ur0.u(t-2e-3) #shift
sig=0.2e-3 #std of gaussian kernel in s
sig2=sig/(t[1]-t[0])
u0 = gaussian_filter1d(u00, sigma=sig2)
u0/=np.amax(np.abs(u0))
u0*=0.5*np.amax(np.abs(u00))

#tuning
BW10_0=500
BW10Func=constant_BW10(BW10_0, requires_grad=False)

plotMaskingDegreeFunc(mdFunc)
plotLatencies(lat0)

In [ ]:
pl.plot(t*1e3, u00, label='unitary response')
pl.plot(t*1e3, u0, label='ur + nPSHT (blur)',  color='orange')
pl.legend(loc='lower right')
pl.title("Unitary response")
pl.xlabel('t (ms)')

Example: two maskers

In [ ]:
json_txt='''{
    "type":"noise-bands", 
    "comment":"created with noiseBandMaskerDesigner", 
    "n_bands":3, 
    "bands":[
        {
            "amplitude":0.02512, 
            "fc_low":2662.5, 
            "fc_high":4162.5
        }, 
        {
            "amplitude":0.1, 
            "fc_low":4800, 
            "fc_high":5200
        }, 
        {
            "amplitude":0.01585, 
            "fc_low":5375, 
            "fc_high":6808.5
        }
    ], 
    "name":"37-2bands-4769-1213-3412-1500-32dB"
}'''

json_txt2='''{
    "type":"noise-bands", 
    "comment":"created with noiseBandMaskerDesigner", 
    "n_bands":1, 
    "bands":{
        "amplitude":0.1, 
        "fc_low":4800, 
        "fc_high":12000
    }, 
    "name":"4-hp-4800Hz"
}'''

json_txt3='''{
    "type":"noise-bands", 
    "comment":"created with noiseBandMaskerDesigner", 
    "n_bands":3, 
    "bands":[
        {
            "amplitude":0.010000000000000002, 
            "fc_low":215.9, 
            "fc_high":2050
        }, 
        {
            "amplitude":0.0001, 
            "fc_low":2050, 
            "fc_high":2950
        }, 
        {
            "amplitude":0.010000000000000002, 
            "fc_low":2950, 
            "fc_high":12000
        }
    ], 
    "name":"2-notch2500-bw900"
}'''


json_txt4='''{
    "type":"noise-bands", 
    "comment":"created with noiseBandMaskerDesigner", 
    "n_bands":3, 
    "bands":[
        {
            "amplitude":0.1, 
            "fc_low":215.9, 
            "fc_high":4450
        }, 
        {
            "amplitude":0.001, 
            "fc_low":4450, 
            "fc_high":5550
        }, 
        {
            "amplitude":0.1, 
            "fc_low":5550, 
            "fc_high":12000
        }
    ], 
    "name":"6-notch5000-bw1100" 
}'''
#stim_struct=json.loads(json_txt)

maskingConditions=MaskingConditions.from_json_strings([json_txt, json_txt2, json_txt3, json_txt4])
#maskingConditions.add_json_strings([json_txt])

In [ ]:
#maskingConditions.get_tensor_lists()
maskingConditions

In [ ]:
pl.figure(figsize=(11,10))
plotMaskingExcitations( BW10Func, maskingConditions)

In [ ]:
maskingConditions.set_amp0_dB(60)

In [ ]:
t=torch.linspace(0.5, 7, 500)*1e-3

#E0=t>1e-3

#E=ExcitationPatterns(t, E0)

E=ExcitationPatterns.GammaExcitation(t, 1, 1.5, 1/(2e-3), 1e-3)

plotExcitationPatterns(E)

pl.figure(figsize=(12, 10))

E.set_masking_model(lat0, BW10Func, maskingConditions, mdFunc, filter_model='gaussian')

#maskingAmount, ex_patterns=E.get_tensors()
plotExcitationPatterns(E, plot_raw_excitation=False)

In [ ]:
#UR
#t2=np.linspace()
urWang=URWang1979shifted.u(t+0.5e-3)
urWangm = URWang1979m.u(t.numpy()-1.5e-3)

dt=t[1]-t[0]
dt=dt.detach().numpy()
ur00=gaussian_filter1d(urWang, 2e-4/dt)
ur0=gaussian_filter1d(urWangm, 2e-4/dt)

pl.figure()
pl.title('Examples of unitary responses')
pl.plot(t*1e3, urWang, label='Wang')
pl.plot(t*1e3, urWangm, label='Wang modified')

pl.plot(t*1e3, ur00, label='Wang + gaussian blur')
pl.plot(t*1e3, ur0, label='Wang m + gaussian blur')
pl.legend()
pl.xlabel('t (ms)')

In [ ]:
#CAP

pl.figure(figsize=(12, 10))
plotSimulatedCAPs(E, ur0)

### Interactive plots

In [ ]:
#NB: withdraw 'manual' for continuous update
@interact_manual( BW0=widgets.IntSlider(value=500, min=100, max=2000, step=50), BW1=widgets.IntSlider(value=500, min=100, max=2000, step=50))
def plotMaskingExcitations_interact(BW0, BW1):
    BW10func0=constant_BW10(BW0)
    BW10func1=constant_BW10(BW1)
    
    pl.figure(figsize=(12, 10))
    axlist=plotMaskingExcitations( BW10func0, maskingConditions)
    plotMaskingExcitations( BW10func1, maskingConditions, axlist=axlist)

In [ ]:

@interact(I0=(30, 100), I1=(30,100), BW0=widgets.IntSlider(value=500, min=100, max=2000, step=50), BW1=widgets.IntSlider(value=500, min=100, max=2000, step=50))
def plotMaskingAmountExcitations_interact(I0, I1, BW0, BW1):
    BW10func0=constant_BW10(BW0)
    BW10func1=constant_BW10(BW1)
    
    pl.figure(figsize=(12, 10))
    
    maskingConditions.set_amp0_dB(I0)
    
    axlist=plotMaskingAmountExcitations( BW10func0, maskingConditions, mdFunc)
    
    maskingConditions.set_amp0_dB(I1)
    axlist=plotMaskingAmountExcitations( BW10func1, maskingConditions, mdFunc, axlist=axlist)

In [ ]:

@interact(I0=(30, 100), I1=(30,100), BW0=widgets.IntSlider(value=500, min=100, max=2000, step=50), BW1=widgets.IntSlider(value=500, min=100, max=2000, step=50))
def plotExcitationPatterns_interact(I0, I1, BW0, BW1):
    BW10func0=constant_BW10(BW0)
    BW10func1=constant_BW10(BW1)
    
    t=torch.linspace(0.5, 7, 500)*1e-3

    E0=t>1e-3

    
    E_0=ExcitationPatterns.copyRaw(E)
    
    E_1=ExcitationPatterns.copyRaw(E)

    pl.figure(figsize=(12, 10))
    
    maskingConditions.set_amp0_dB(I0)
    
    E_0.set_masking_model(lat0, BW10func0, maskingConditions, mdFunc, filter_model='gaussian')
    axlist=plotExcitationPatterns(E_0)
    
    maskingConditions.set_amp0_dB(I1)
    
    E_1.set_masking_model(lat0, BW10func1, maskingConditions, mdFunc, filter_model='gaussian')
    axlist=plotExcitationPatterns(E_1, axlist=axlist)
    

In [ ]:

@interact(I0=(30, 100), I1=(30,100), BW0=widgets.IntSlider(value=500, min=100, max=2000, step=50), BW1=widgets.IntSlider(value=500, min=100, max=2000, step=50))
def plotSimulatedCAPs_interact(I0, I1, BW0, BW1):
    BW10func0=constant_BW10(BW0)
    BW10func1=constant_BW10(BW1)
    
    t=torch.linspace(0.5, 7, 500)*1e-3


    
    E_0=ExcitationPatterns.copyRaw(E)
    E_1=ExcitationPatterns.copyRaw(E)

    pl.figure(figsize=(12, 10))
    
    maskingConditions.set_amp0_dB(I0)
    
    E_0.set_masking_model(lat0, BW10func0, maskingConditions, mdFunc, filter_model='gaussian')
    axlist=plotSimulatedCAPs(E_0, ur0, shift=0.7e-3)
    
    maskingConditions.set_amp0_dB(I1)
    
    E_1.set_masking_model(lat0, BW10func1, maskingConditions, mdFunc, filter_model='gaussian')
    axlist=plotSimulatedCAPs(E_1, ur0, axlist=axlist, shift=0.7e-3)
    